In [14]:
# -*- coding: utf-8 -*-
# -*- coding: utf-8 -*-
import os
import sys
import dataiku
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.externals import joblib
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer
from my_preprocessing import preprocessing

############################## Your params here ################################

my_input_dataset_name = "to_score"
my_output_folder_id = "03yT6tmA"
output_dataset_name = "mymodel_scored"

target_name = 'target'
## Variables to exclude
exclude_vars = ['customer_id','revenue','original_dataset','ip','birth_parsed']


#################################################################################

In [15]:
# Read recipe inputs
train_test = dataiku.Dataset(my_input_dataset_name)
df = train_test.get_dataframe()
df_base = train_test.get_dataframe()

# Write recipe outputs
mypythonmodelfolder = dataiku.Folder(my_output_folder_id)
mypythonmodelfolder_info = mypythonmodelfolder.get_info()
model_scikit = mypythonmodelfolder_info.get('path')

schema = joblib.load(os.path.join(model_scikit, 'schema.pkl'))
model0 = joblib.load(os.path.join(model_scikit, '0.pkl'))

In [16]:
features_num =[]
features_cat =[]
cat =['string','boolean']
for s in train_test.read_schema():
    var = s['name']
    type_=s['type']
    if var not in exclude_vars and var <> target_name:
        if type_ in cat:
            features_cat.append(var)
        else:
            features_num.append(var)

features_all = features_num + features_cat

rescale_features = {}
for c in features_num:
    rescale_features[c]=u'AVGSTD'

In [17]:
df[features_cat].fillna('unknown',inplace=True)

In [18]:
for feature in features_cat:
    df[feature] = df[feature].apply(preprocessing.coerce_to_unicode)

for feature in features_num:
    if df[feature].dtype == np.dtype('M8[ns]'):
        df[feature] = datetime_to_epoch(df[feature])
    else:
        df[feature] = df[feature].astype('double')

In [19]:
DUMMY_VALUES = preprocessing.select_dummy_values(df, features_cat)

In [20]:
preprocessing.dummy_encode_dataframe(DUMMY_VALUES,df)

Dummy-encoded feature gender
Dummy-encoded feature ip_country_code
Dummy-encoded feature campain
Dummy-encoded feature Dataset
Dummy-encoded feature end_with_d
Dummy-encoded feature ip_geopoint


In [21]:
preprocessing.rescale_features_(rescale_features,df)

Rescaled Age
Rescaled pages_visited
Rescaled price_first_item_purchased


In [22]:
for c in schema['features_all']:
    if c not in list(df.columns):
        print (c)
        df[c]=0

ip_country_code_value_NG
ip_country_code_value_BA
ip_country_code_value_GT
ip_country_code_value_BO
ip_country_code_value_SC
ip_country_code_value_PY
ip_country_code_value_EE
ip_country_code_value_BN
ip_country_code_value_GH
ip_country_code_value_AL
ip_country_code_value_CY
ip_country_code_value_LB
ip_country_code_value_AM
ip_country_code_value_OM
ip_country_code_value_HN
Dataset_value_last_year
end_with_d_value_True
ip_geopoint_value_POINT(-122.3129 47.5413)
ip_geopoint_value_POINT(100.4667 13.75)
ip_geopoint_value_POINT(-74.0628 4.6492)
ip_geopoint_value_POINT(117.2808 31.8639)
ip_geopoint_value_POINT(30.5238 50.4547)
ip_geopoint_value_POINT(9.1895 45.4643)
ip_geopoint_value_POINT(-5 32)
ip_geopoint_value_POINT(12.5113 41.8919)
ip_geopoint_value_POINT(9 34)
ip_geopoint_value_POINT(-118.244 34.0544)
ip_geopoint_value_POINT(-87.6517 41.8483)
ip_geopoint_value_POINT(120.9721 14.5955)
ip_geopoint_value_POINT(31.2859 30.0771)
ip_geopoint_value_POINT(25 46)
ip_geopoint_value_POINT(108.9286

In [25]:
X = df[schema['features_all']].values

scores0 = model0.predict(X)
df['prediction_custom_model'] = scores0
df_base['prediction_custom_model'] = scores0
df_base.replace({"prediction_custom_model":     {1: "high_revenue", 0: "low_revenue"}}, inplace=True)

In [26]:
df_base.head()

,customer_id,Age,price_first_item_purchased,gender,revenue,Dataset,end_with_d,ip,ip_geopoint,ip_country_code,pages_visited,campain,target,prediction_custom_model
0,88789ebaf5a,29,10.0,F,NaN,to_score,False,41.189.149.136,NaN,NaN,7.0,False,NaN,low_revenue
1,49587b0b4aa,35,28.0,F,NaN,to_score,False,149.141.47.50,POINT(-97.822 37.751),US,8.0,False,NaN,high_revenue
2,5cdbeab2fba,120,42.0,F,NaN,to_score,False,161.35.97.107,POINT(-97.822 37.751),US,5.0,True,NaN,low_revenue
3,069e998177a,27,42.0,F,NaN,to_score,False,180.56.63.96,POINT(139.6489 35.8569),JP,6.0,False,NaN,high_revenue
4,8bb69a2e66a,120,102.0,F,NaN,to_score,False,115.112.71.94,POINT(73.8667 18.5333),IN,6.0,True,NaN,low_revenue


In [27]:
# Write recipe outputs
score_custom = dataiku.Dataset(output_dataset_name)
score_custom.write_with_schema(df)

4938 rows successfully written (6VI4sYDSSk)
